In [1]:
!pip install BeautifulSoup4

     |████████████████████████████████| 112kB 7.4MB/s eta 0:00:01


In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
- 

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


# Function for getting lat and long from geocode

In [5]:
def geo(dataframe): 
    geolocator = Nominatim(user_agent="singapore_explorer")
    from geopy.extra.rate_limiter import RateLimiter
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#RateLimiter allows to perform bulk operations while gracefully handling error responses and adding delays when needed.

#In the example below a delay of 1 second (min_delay_seconds=1) will be added between each pair of geolocator.geocode calls;
#all geopy.exc.GeocoderServiceError exceptions will be retried (up to max_retries times):
    dataframe['location'] = dataframe['location_of_centre'].apply(geocode)

    dataframe['point'] = dataframe['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    return dataframe.head()

# Function for getting lat and long separately 

In [6]:
def split(data):
    long = []
    lat = []
    for x,y,z in data.point:
            long.append(x)
            lat.append(y)
    frame = pd.DataFrame({'Latitude':long , 'Longitude':lat})
    
    return frame

# Function for merging dataframes 

In [7]:
def together(data1, data2): 
    data1 = data1.merge(data2, on=data1.location_of_centre)
    data1 = data1.drop('key_0', axis=1)
    return data1

# Get Postalcodes from wikipedia

In [9]:
url = 'https://en.wikipedia.org/wiki/Postal_codes_in_Singapore'
results = requests.get(url).text
re = BeautifulSoup(results, 'html.parser')
table = re.find('table')

# Create empty dataframe 

In [10]:
column_names = ['Postal_District', 'Postalcode', 'Neighborhood']
frame = pd.DataFrame(columns = column_names)
frame

,Postal_District,Postalcode,Neighborhood


In [11]:
for tr in table.findAll("tr"):
    row = []
    for td in tr.findAll('td'):
        row.append(td.text.strip())
    if len(row) == 3:
        frame.loc[len(frame)] = row 

In [12]:
frame.isnull().sum() #check for bad rows

Postal_District    0
Postalcode         0
Neighborhood       0
dtype: int64

In [134]:
frame.head()

,Postal_District,Postalcode,Neighborhood
0,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
1,02,"07, 08","Anson, Tanjong Pagar"
2,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
3,04,"09, 10","Telok Blangah, Harbourfront"
4,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [13]:
new_df = pd.DataFrame(frame.Neighborhood.str.split(',').tolist()).stack().reset_index()
new_df.columns = ['one', 'two', 'location_of_centre']
new = new_df.drop(['one','two'], axis=1)
new

,location_of_centre
0,Raffles Place
1,Cecil
2,Marina
3,People's Park
4,Anson
5,Tanjong Pagar
6,Bukit Merah
7,Queenstown
8,Tiong Bahru
9,Telok Blangah


In [14]:
new_1 = new[['location_of_centre']] + ', Singapore' #adding 'Singapore' so as to locate on geolocator
new_1.head()

,location_of_centre
0,"Raffles Place, Singapore"
1,"Cecil, Singapore"
2,"Marina, Singapore"
3,"People's Park, Singapore"
4,"Anson, Singapore"


In [15]:
geo(new_1)

,location_of_centre,location,point
0,"Raffles Place, Singapore","(Raffles Place, Malacca Street, Downtown Core,...","(1.2835416999999998, 103.85146023266938, 0.0)"
1,"Cecil, Singapore","(Cecil Street, Downtown Core, Singapore, Centr...","(1.276564, 103.8469576, 0.0)"
2,"Marina, Singapore","(Singapore, Central, 178957, Singapore, (1.290...","(1.2904753, 103.8520359, 0.0)"
3,"People's Park, Singapore","(People's Park, Upper Cross Street, Chinatown,...","(1.2858105, 103.8441598, 0.0)"
4,"Anson, Singapore","(Anson Road, Radin Mas, Bukit Merah, Singapore...","(1.2718055, 103.843321, 0.0)"


In [16]:
new_1.tail()

,location_of_centre,location,point
70,"Upper Thomson, Singapore","(Upper Thomson, Upper Thomson Road, Bishan, Si...","(1.354498, 103.83282055067511, 0.0)"
71,"Springleaf, Singapore","(Springleaf, Upper Thomson Road, Yishun, Singa...","(1.398315, 103.81805726106943, 0.0)"
72,"Yishun, Singapore","(Yishun, Yishun Avenue 2, Yishun, Northwest, 7...","(1.4293839, 103.8350282, 0.0)"
73,"Sembawang, Singapore","(Sembawang, Sembawang Way, Sembawang, Northwes...","(1.4490928, 103.8200555, 0.0)"
74,"Seletar, Singapore","(Seletar, Singapore, Central, Singapore, (1.40...","(1.4098488, 103.8773789, 0.0)"


In [17]:
new_1.shape

(75, 3)

In [18]:
#new_1.isnull()

In [19]:
new_1.iloc[12:16, :]

,location_of_centre,location,point
12,"Hong Leong Garden, Singapore",None,None
13,"Clementi New Town, Singapore","(Clementi, Southwest, Singapore, (1.3140256, 1...","(1.3140256, 103.7624098, 0.0)"
14,"High Street, Singapore","(Parliament Place, Downtown Core, Singapore, C...","(1.289648, 103.8506924, 0.0)"
15,"Beach Road (part), Singapore",None,None


In [20]:
new_1 = new_1.dropna(subset = ['location', 'point'])

In [21]:
new_1.reset_index()

,index,location_of_centre,location,point
0,0,"Raffles Place, Singapore","(Raffles Place, Malacca Street, Downtown Core,...","(1.2835416999999998, 103.85146023266938, 0.0)"
1,1,"Cecil, Singapore","(Cecil Street, Downtown Core, Singapore, Centr...","(1.276564, 103.8469576, 0.0)"
2,2,"Marina, Singapore","(Singapore, Central, 178957, Singapore, (1.290...","(1.2904753, 103.8520359, 0.0)"
3,3,"People's Park, Singapore","(People's Park, Upper Cross Street, Chinatown,...","(1.2858105, 103.8441598, 0.0)"
4,4,"Anson, Singapore","(Anson Road, Radin Mas, Bukit Merah, Singapore...","(1.2718055, 103.843321, 0.0)"
5,5,"Tanjong Pagar, Singapore","(Tanjong Pagar, Underground Path from Tanjong ...","(1.2765707, 103.845848, 0.0)"
6,6,"Bukit Merah, Singapore","(Bukit Merah, Singapore, Central, Singapore, (...","(1.2704395, 103.82831840176755, 0.0)"
7,7,"Queenstown, Singapore","(Queenstown, Commonwealth Avenue, Nepal Hill, ...","(1.2946235, 103.8060454, 0.0)"
8,8,"Tiong Bahru, Singapore","(Tiong Bahru, Singapore, Central, 164019, Sing...","(1.2861968, 103.8257646, 0.0)"
9,9,"Telok Blangah, Singapore","(Telok Blangah, Telok Blangah Road, Radin Mas,...","(1.2705858, 103.8098632, 0.0)"


In [22]:
new_2 = split(new_1)

In [52]:
new_lst = together(new_1,new_2)

In [53]:
city = 'Singapore'

geolocator = Nominatim(user_agent="sin_explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


In [54]:
CLIENT_ID = 'WFK1SF3GTUYLQUHZ15BDG1XOEBSVEAMLQ3DMUWKTJAZOCFGH' # your Foursquare ID
CLIENT_SECRET = 'G1ZQ50ZJ5XIOFMYYCOD1OWG5BCMGR4GDA0ZYTIJLB1XKLD2A' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
sin_venues = getNearbyVenues(names=new_lst['location'],
                                   latitudes=new_lst['Latitude'],
                                   longitudes=new_lst['Longitude']
                                  )

Raffles Place, Malacca Street, Downtown Core, Singapore, Central, 049317, Singapore
Cecil Street, Downtown Core, Singapore, Central, 79903, Singapore
Singapore, Central, 178957, Singapore
People's Park, Upper Cross Street, Chinatown, Outram, Singapore, Central, 059805, Singapore
Anson Road, Radin Mas, Bukit Merah, Singapore, Central, 089055, Singapore
Tanjong Pagar, Underground Path from Tanjong Pagar MRT, Downtown Core, Singapore, Central, 069119, Singapore
Bukit Merah, Singapore, Central, Singapore
Queenstown, Commonwealth Avenue, Nepal Hill, Queenstown, Singapore, Central, 149729, Singapore
Tiong Bahru, Singapore, Central, 164019, Singapore
Telok Blangah, Telok Blangah Road, Radin Mas, Bukit Merah, Southwest, 109028, Singapore
HarbourFront, Harbourfront Avenue, Radin Mas, Bukit Merah, Singapore, Central, 098499, Singapore
Pasir Panjang, West Coast Highway, Queenstown, Southwest, 109024, Singapore
Clementi, Southwest, Singapore
Parliament Place, Downtown Core, Singapore, Central, 179

In [57]:
sin_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"(Raffles Place, Malacca Street, Downtown Core,...",1.283542,103.85146,The Fullerton Bay Hotel,1.283878,103.853314,Hotel
1,"(Raffles Place, Malacca Street, Downtown Core,...",1.283542,103.85146,CITY Hot Pot Shabu shabu,1.284173,103.851585,Hotpot Restaurant
2,"(Raffles Place, Malacca Street, Downtown Core,...",1.283542,103.85146,Virgin Active,1.284608,103.850815,Gym / Fitness Center
3,"(Raffles Place, Malacca Street, Downtown Core,...",1.283542,103.85146,CULINARYON,1.284876,103.850933,Comfort Food Restaurant
4,"(Raffles Place, Malacca Street, Downtown Core,...",1.283542,103.85146,1-Altitude,1.284794,103.851151,Cocktail Bar


In [58]:
sin_venues.Neighborhood = sin_venues.Neighborhood.astype(str)


In [59]:
sin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"112 Katong, East Coast Road, Marine Parade, Southeast, 428802, Singapore",74,74,74,74,74,74
"2450, Ang Mo Kio Avenue 8, Ang Mo Kio, Singapore, Central, 569811, Singapore",42,42,42,42,42,42
"Amber Road, Marine Parade, Southeast, 489886, Singapore",48,48,48,48,48,48
"Anson Road, Radin Mas, Bukit Merah, Singapore, Central, 089055, Singapore",63,63,63,63,63,63
"Balestier, Singapore, Central, 329842, Singapore",49,49,49,49,49,49
"Bedok, New Upper Changi Road, Simpang Bedok, Bedok, Southeast, 460062, Singapore",60,60,60,60,60,60
"Bishan, Bishan Place, Bishan, Singapore, Central, 579837, Singapore",46,46,46,46,46,46
"Braddell, Lorong 2 Toa Payoh, Toa Payoh, Singapore, Central, 319758, Singapore",42,42,42,42,42,42
"Bukit Merah, Singapore, Central, Singapore",5,5,5,5,5,5


# Check how many venues were returned for each neighborhood

In [60]:
# one hot encoding
sin = pd.get_dummies(sin_venues[['Venue Category']], prefix="", prefix_sep="")
#separates the different venues into columns

# add neighborhood column back to dataframe
sin['Neighborhood'] =sin_venues['Neighborhood'] 
sin['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [sin.columns[-1]] + list(sin.columns[:-1])
sin = sin[fixed_columns]


In [61]:
sin['Neighborhood'] = sin[['Neighborhood']].astype(str) #original data was in geopy.location.Location

In [62]:
sin_grouped = sin.groupby(sin['Neighborhood']).mean().reset_index()

In [63]:
sin_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Border Crossing,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Canal,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Gym,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Farm,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,History Museum,Hobby Shop,Hong Kong Restaurant,Hospital,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Lounge,Macanese Restaurant,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Parking,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Travel Lounge,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"112 Katong, East Coast Road, Marine Parade, So...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040541,0.013514,0.013514,0.000000,0.0,0.000000,0.067568,0.00000,0.040541,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.013514,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0

# Let's print each neighborhood along with the top 10 most common venues¶

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sin_grouped['Neighborhood']

for ind in np.arange(sin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"112 Katong, East Coast Road, Marine Parade, So...",Chinese Restaurant,Bakery,Hotel,Noodle House,Indian Restaurant,Japanese Restaurant,Coffee Shop,Asian Restaurant,Massage Studio,Bar
1,"2450, Ang Mo Kio Avenue 8, Ang Mo Kio, Singapo...",Coffee Shop,Food Court,Dessert Shop,Supermarket,Sandwich Place,Bubble Tea Shop,Fast Food Restaurant,Japanese Restaurant,Fried Chicken Joint,Chinese Restaurant
2,"Amber Road, Marine Parade, Southeast, 489886, ...",Hotel,Café,Japanese Restaurant,Indian Restaurant,Bar,Chinese Restaurant,Flower Shop,Ice Cream Shop,Food Court,Supermarket
3,"Anson Road, Radin Mas, Bukit Merah, Singapore,...",Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Soup Place,Soba Restaurant,Bakery,Sushi Restaurant,Salad Place
4,"Balestier, Singapore, Central, 329842, Singapore",Chinese Restaurant,Hotel,Fast Food Restaurant,Food Court,Noodle House,Bakery,Asian Restaurant,Indian Restaurant,Shopping Mall,General Entertainment


In [66]:
# set number of clusters
kclusters = 8

sin_grouped_clustering = sin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 6, 0, 0, 0, 0, 0], dtype=int32)

In [67]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

In [68]:
for x in neighborhoods_venues_sorted['Neighborhood']:
    print (x)
    break

112 Katong, East Coast Road, Marine Parade, Southeast, 428802, Singapore


In [69]:
#new_lst.head()

In [70]:
neighborhoods_venues_sorted.Neighborhood = neighborhoods_venues_sorted.Neighborhood.astype(str)

In [71]:
new_again = new_lst

In [72]:
new_again['location'] = new_again['location'].astype(str)

In [73]:
new_again.sort_values(by=['location'], ascending=True)

,location_of_centre,location,point,Latitude,Longitude
37,"Katong, Singapore","112 Katong, East Coast Road, Marine Parade, So...","(1.3052327, 103.90505238485225, 0.0)",1.305233,103.905052
53,"Ang Mo Kio, Singapore","2450, Ang Mo Kio Avenue 8, Ang Mo Kio, Singapo...","(1.3700733, 103.8495157, 0.0)",1.370073,103.849516
39,"Amber Road, Singapore","Amber Road, Marine Parade, Southeast, 489886, ...","(1.303121, 103.900556, 0.0)",1.303121,103.900556
4,"Anson, Singapore","Anson Road, Radin Mas, Bukit Merah, Singapore,...","(1.2718055, 103.843321, 0.0)",1.271805,103.843321
30,"Balestier, Singapore","Balestier, Singapore, Central, 329842, Singapore","(1.326226, 103.8473149, 0.0)",1.326226,103.847315
40,"Bedok, Singapore","Bedok, New Upper Changi Road, Simpang Bedok, B...","(1.3239765, 103.930216, 0.0)",1.323976,103.930216
52,"Bishan, Singapore","Bishan, Bishan Place, Bishan, Singapore, Centr...","(1.3509859, 103.84825507492937, 0.0)",1.350986,103.848255
34,"Braddell, Singapore","Braddell, Lorong 2 Toa Payoh, Toa Payoh, Singa...","(1.3400707, 103.84675837096586, 0.0)",1.340071,103.846758
6,"Bukit Merah, Singapore","Bukit Merah, Singapore, Central, Singapore","(1.2704395, 103.82831840176755, 0.0)",1.270439,103.828318
61,"Bukit Panjang, Singapore","Bukit Panjang, Exit D, Bukit Panjang, Northwes...","(1.378629, 103.7621358, 0.0)",1.378629,103.762136


In [74]:
new_again = new_again.rename(columns={'location':'Neighborhood'})

In [75]:
new_merge = new_again.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on=new_again['Neighborhood'])

In [76]:
new_merge.head()

,location_of_centre,Neighborhood,point,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Raffles Place, Singapore","Raffles Place, Malacca Street, Downtown Core, ...","(1.2835416999999998, 103.85146023266938, 0.0)",1.283542,103.851460,0.0,Hotel,Salad Place,Italian Restaurant,Sandwich Place,French Restaurant,Gym,Cocktail Bar,Coffee Shop,Food Court,Café
1,"Cecil, Singapore","Cecil Street, Downtown Core, Singapore, Centra...","(1.276564, 103.8469576, 0.0)",1.276564,103.846958,0.0,Coffee Shop,Japanese Restaurant,Hotel,Café,Ramen Restaurant,Cocktail Bar,Bakery,Restaurant,Gym / Fitness Center,Bar
2,"Marina, Singapore","Singapore, Central, 178957, Singapore","(1.2904753, 103.8520359, 0.0)",1.290475,103.852036,0.0,Hotel,Concert Hall,Coffee Shop,Restaurant,Shopping Mall,Italian Restaurant,Music Venue,Monument / Landmark,Performing Arts Venue,Theater
3,"People's Park, Singapore","People's Park, Upper Cross Street, Chinatown, ...","(1.2858105, 103.8441598, 0.0)",1.285810,103.844160,0.0,Chinese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Food Court,Hostel,Spa,Japanese Restaurant,Café,Thai Restaurant,Gym
4,"Anson, Singapore","Anson Road, Radin Mas, Bukit Merah, Singapore,...","(1.2718055, 103.843321, 0.0)",1.271805,103.843321,0.0,Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Soup Place,Soba Restaurant,Bakery,Sushi Restaurant,Salad Place


In [77]:
new_merge = new_merge.dropna(subset = ['Cluster Labels'])

In [78]:
new_merge[['Cluster Labels']] = new_merge[['Cluster Labels']].astype(int)

In [79]:
from folium import plugins

In [80]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_merge['Latitude'], new_merge['Longitude'], new_merge['Neighborhood'], new_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# examine cluster 0 

In [81]:
new_merge.loc[new_merge['Cluster Labels'] == 0, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Raffles Place, Malacca Street, Downtown Core, ...",0,Hotel,Salad Place,Italian Restaurant,Sandwich Place,French Restaurant,Gym,Cocktail Bar,Coffee Shop,Food Court,Café
1,"Cecil Street, Downtown Core, Singapore, Centra...",0,Coffee Shop,Japanese Restaurant,Hotel,Café,Ramen Restaurant,Cocktail Bar,Bakery,Restaurant,Gym / Fitness Center,Bar
2,"Singapore, Central, 178957, Singapore",0,Hotel,Concert Hall,Coffee Shop,Restaurant,Shopping Mall,Italian Restaurant,Music Venue,Monument / Landmark,Performing Arts Venue,Theater
3,"People's Park, Upper Cross Street, Chinatown, ...",0,Chinese Restaurant,Hotel,Vegetarian / Vegan Restaurant,Food Court,Hostel,Spa,Japanese Restaurant,Café,Thai Restaurant,Gym
4,"Anson Road, Radin Mas, Bukit Merah, Singapore,...",0,Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Soup Place,Soba Restaurant,Bakery,Sushi Restaurant,Salad Place
5,"Tanjong Pagar, Underground Path from Tanjong P...",0,Japanese Restaurant,Coffee Shop,Bakery,Hotel,Gym / Fitness Center,Café,Salad Place,Cocktail Bar,Spanish Restaurant,Restaurant
6,"Bukit Merah, Singapore, Central, Singapore",0,Food Court,Playground,Coffee Shop,Hotel,Harbor / Marina,French Restaurant,Food Truck,Food Stand,Fried Chicken Joint,Filipino Restaurant
7,"Queenstown, Commonwealth Avenue, Nepal Hill, Q...",0,Food Court,Noodle House,Chinese Restaurant,Thai Restaurant,Spa,Train Station,Café,Seafood Restaurant,Stadium,Bus Line
9,"Telok Blangah, Telok Blangah Road, Radin Mas, ...",0,Coffee Shop,Chinese Restaurant,Food Court,Cupcake Shop,Market,Convenience Store,Recreation Center,Club House,Noodle House,Shopping Mall
10,"HarbourFront, Harbourfront Avenue, Radin Mas, ...",0,Chinese Restaurant,Japanese Restaurant,Clothing Store,Toy / Game Store,Fast Food Restaurant,Multiplex,Café,Bakery,Malay Restaurant,Shoe Store


# Examine cluster 1

In [82]:
new_merge.loc[new_merge['Cluster Labels'] == 1, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,"Tuas, Southwest, Singapore",1,Bar,Coffee Shop,Food Court,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Yoga Studio,Fish & Chips Shop


# Examine cluster 2

In [83]:
new_merge.loc[new_merge['Cluster Labels'] == 2, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,"Loyang, Southeast, 508923, Singapore",2,Smoke Shop,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand


# Examine cluster 3

In [84]:
new_merge.loc[new_merge['Cluster Labels'] == 3, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,"Lim Chu Kang, Southwest, Singapore",3,Zoo Exhibit,Farm,Fish & Chips Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand


# Examine cluster 4

In [85]:
new_merge.loc[new_merge['Cluster Labels'] == 4, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Tengah, Southwest, Singapore",4,Cosmetics Shop,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand


# Examine cluster 5

In [86]:
new_merge.loc[new_merge['Cluster Labels'] == 5, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Bukit Timah Hill, Northwest, 589333, Singapore",5,Trail,Mountain,Rest Area,Scenic Lookout,Yoga Studio,Food & Drink Shop,Fishing Spot,Flea Market,Flower Shop,Food
60,"Dairy Farm, Northwest, 677621, Singapore",5,Trail,Public Art,Rest Area,Nature Preserve,Bike Trail,Gym,Parking,Discount Store,Dive Bar,Fried Chicken Joint


# Examine cluster 6

In [87]:
new_merge.loc[new_merge['Cluster Labels'] == 6, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Tiong Bahru, Singapore, Central, 164019, Singa...",6,Chinese Restaurant,Food Court,Bakery,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Hainan Restaurant,Sandwich Place,Café,Noodle House
21,"The Cairnhill, Newton, Singapore, Central, Sin...",6,Hotel,Pub,Wine Bar,Filipino Restaurant,Chinese Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Frozen Yogurt Shop
23,"Le Nouvel Ardmore, Newton, Singapore, Central,...",6,Hotel,Hotel Bar,Japanese Restaurant,Chinese Restaurant,Spa,Gym,Sushi Restaurant,Steakhouse,Buffet,Café
30,"Balestier, Singapore, Central, 329842, Singapore",6,Chinese Restaurant,Hotel,Fast Food Restaurant,Food Court,Noodle House,Bakery,Asian Restaurant,Indian Restaurant,Shopping Mall,General Entertainment
35,"Geylang, Southeast, Singapore",6,Chinese Restaurant,Noodle House,Asian Restaurant,Food Court,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Grocery Store,Cantonese Restaurant,Basketball Court,Seafood Restaurant
69,"Springleaf, Upper Thomson Road, Yishun, Singap...",6,Chinese Restaurant,Asian Restaurant,Café,Thai Restaurant,Indian Restaurant,Supermarket,German Restaurant,Massage Studio,Park,Convenience Store


In [88]:
# Examine cluster 7 

In [89]:
new_merge.loc[new_merge['Cluster Labels'] == 7, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"Changi, Southeast, Singapore",7,Airport,Spa,Rest Area,Border Crossing,Hotel,Yoga Studio,Food Stand,Flea Market,Flower Shop,Food


# Examine cluster 8 

In [90]:
new_merge.loc[new_merge['Cluster Labels'] == 8, new_merge.columns[[1] + list(range(5, new_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Import data for food centres not found in foursquare 

In [91]:
path = r'list-of-government-markets-hawker-centres.csv'
df = pd.read_csv(path)
df.head()

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0


In [92]:
hawker = df['type_of_centre'] == 'MHC'

In [93]:
df_MHC = df[hawker]
df_MHC.shape

(77, 7)

In [94]:
geo(df_MHC)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls,location,point
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62,None,None
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95,"(Upper Bukit Timah Road, Bukit Panjang, Northw...","(1.3583405, 103.7675074, 0.0)"
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63,"(31, Commonwealth Crescent, Nepal Hill, Queens...","(1.3068116, 103.8004053, 0.0)"
10,Geylang Serai Market,"Geylang Serai, S(402001)",MHC,Government,365,63,302,"(Geylang Serai, Geylang, Southeast, 400416, Si...","(1.3171786, 103.8973614, 0.0)"
11,Golden Mile Food Centre,"505, Beach Road, S(199583)",MHC,Government,223,112,111,"(505, Beach Road, Kallang, Singapore, Central,...","(1.303137, 103.8639373, 0.0)"


In [95]:
df_MHC = df_MHC.drop(['owner', 'no_of_cooked_food_stalls','no_of_mkt_produce_stalls', 'no_of_stalls'],axis=1)

In [96]:
df_MHC = df_MHC.dropna(subset =['location', 'point'])

In [97]:
data2 = split(df_MHC)

In [98]:
df_MHC_1 = together(df_MHC, data2)

In [99]:
# instantiate a feature group for the incidents in the dataframe
hawker_map = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_MHC_1.Latitude, df_MHC_1.Longitude):
    hawker_map.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
new_map = map_clusters.add_child(hawker_map)
new_map

# Import data for no of professionals

In [100]:
path = r'resident-working-persons-aged-15-years-and-over-by-planning-area-and-occupation.csv'
jobs = pd.read_csv(path)
jobs.head()

,year,level_1,level_2,level_3,value
0,2015,Total,Total,Ang Mo Kio,101.2
1,2015,Total,Total,Bedok,150.3
2,2015,Total,Total,Bishan,49.5
3,2015,Total,Total,Bukit Batok,75.6
4,2015,Total,Total,Bukit Merah,82.2


In [101]:
total = jobs['level_1'] == 'Total'
total_occupation = jobs[total]
total_occupation.head()

,year,level_1,level_2,level_3,value
0,2015,Total,Total,Ang Mo Kio,101.2
1,2015,Total,Total,Bedok,150.3
2,2015,Total,Total,Bishan,49.5
3,2015,Total,Total,Bukit Batok,75.6
4,2015,Total,Total,Bukit Merah,82.2


# Change the value in total_occupation to actual value (in thousands)

In [102]:
total_occupation['value'] = total_occupation['value'] * 1000

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
total_occupation_1 = total_occupation.drop(['year','level_2'], axis=1)

In [104]:
total_occupation_1.columns = ['Jobs', 'location_of_centre', 'No of Population']
total_occupation_1['location_of_centre'] = total_occupation_1['location_of_centre'] + ' , Singapore'

In [105]:
total_occupation_1

,Jobs,location_of_centre,No of Population
0,Total,"Ang Mo Kio , Singapore",101200.0
1,Total,"Bedok , Singapore",150300.0
2,Total,"Bishan , Singapore",49500.0
3,Total,"Bukit Batok , Singapore",75600.0
4,Total,"Bukit Merah , Singapore",82200.0
5,Total,"Bukit Panjang , Singapore",80100.0
6,Total,"Bukit Timah , Singapore",36800.0
7,Total,"Choa Chu Kang , Singapore",97200.0
8,Total,"Clementi , Singapore",45900.0
9,Total,"Geylang , Singapore",61400.0


In [106]:
geo(total_occupation_1)

,Jobs,location_of_centre,No of Population,location,point
0,Total,"Ang Mo Kio , Singapore",101200.0,"(2450, Ang Mo Kio Avenue 8, Ang Mo Kio, Singap...","(1.3700733, 103.8495157, 0.0)"
1,Total,"Bedok , Singapore",150300.0,"(Bedok, New Upper Changi Road, Simpang Bedok, ...","(1.3239765, 103.930216, 0.0)"
2,Total,"Bishan , Singapore",49500.0,"(Bishan, Bishan Place, Bishan, Singapore, Cent...","(1.3509859, 103.84825507492937, 0.0)"
3,Total,"Bukit Batok , Singapore",75600.0,"(Bukit Batok, Bukit Batok Central Link, Bukit ...","(1.3490572, 103.7495906, 0.0)"
4,Total,"Bukit Merah , Singapore",82200.0,"(Bukit Merah, Singapore, Central, Singapore, (...","(1.2704395, 103.82831840176755, 0.0)"


In [107]:
total_occupation_1 = total_occupation_1.dropna(subset=['point', 'location'])

In [108]:
new = split(total_occupation_1)

In [109]:
total_occupation_1 = total_occupation_1.merge(new, on=total_occupation_1.location_of_centre)
total_occupation_1.head()

,key_0,Jobs,location_of_centre,No of Population,location,point,Latitude,Longitude
0,"Ang Mo Kio , Singapore",Total,"Ang Mo Kio , Singapore",101200.0,"(2450, Ang Mo Kio Avenue 8, Ang Mo Kio, Singap...","(1.3700733, 103.8495157, 0.0)",1.370073,103.849516
1,"Bedok , Singapore",Total,"Bedok , Singapore",150300.0,"(Bedok, New Upper Changi Road, Simpang Bedok, ...","(1.3239765, 103.930216, 0.0)",1.323976,103.930216
2,"Bishan , Singapore",Total,"Bishan , Singapore",49500.0,"(Bishan, Bishan Place, Bishan, Singapore, Cent...","(1.3509859, 103.84825507492937, 0.0)",1.350986,103.848255
3,"Bukit Batok , Singapore",Total,"Bukit Batok , Singapore",75600.0,"(Bukit Batok, Bukit Batok Central Link, Bukit ...","(1.3490572, 103.7495906, 0.0)",1.349057,103.749591
4,"Bukit Merah , Singapore",Total,"Bukit Merah , Singapore",82200.0,"(Bukit Merah, Singapore, Central, Singapore, (...","(1.2704395, 103.82831840176755, 0.0)",1.270439,103.828318


In [132]:
total_occupation_1['No of Population'].min()

'101200.0'

In [133]:
total_occupation_1['No of Population'].max()

'97200.0'

In [110]:
#import csv
#import json
#from collections import OrderedDict

In [112]:
#!pip install geojson

In [113]:
#import geojson
#from geojson import Feature, FeatureCollection, Point

In [125]:
total_occupation_1['location'] = total_occupation_1['location'].astype(str)

In [126]:
# columns used for constructing geojson object
#features = total_occupation_1.apply(
#    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
#    axis=1).tolist()

# all the other columns used as properties
#properties = total_occupation_1.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
#feature_collection = FeatureCollection(features=features, properties=properties)

In [127]:
#with open('wl.geojson', 'w', encoding='utf-8') as f:
#    json.dump(feature_collection, f, ensure_ascii=False)

In [ ]:
#total_occupation_1.location = total_occupation_1.location.astype(str)

In [120]:
total_occupation_1['No of Population'] = total_occupation_1['No of Population'].astype(str)

In [121]:
# instantiate a feature group for the incidents in the dataframe


real_map = new_map
tired = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(total_occupation_1.Latitude, total_occupation_1.Longitude):
    tired.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(total_occupation_1.Latitude)
longitudes = list(total_occupation_1.Longitude)
label1 = list(total_occupation_1['No of Population'])
#label2 = list(total_occupation_1['location_of_centre'])

In [122]:
for lat, lng, label1 in zip(latitudes, longitudes, label1):
              folium.Marker([lat, lng], popup=label1).add_to(real_map)    
    
# add incidents to map
real_map.add_child(tired)